Colm Mooney - 20325583

Goal: Understand how different audio features used with Hidden Markov Models affects Speech Recognition task.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install -q keras

In [3]:
!pip install hmmlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 2.7 MB/s eta 0:00:00


In [4]:
import keras

In [ ]:
#Code works, just isn't needed currently after doing it once (Splits code in dataset to 80:20, 80% for training, 20% for testing.)
#import os
#import shutil

# Define paths for train and test directories
#data_directory = '/content/drive/MyDrive/dataset'
#train_directory = os.path.join(data_directory, 'train_data')
#test_directory = os.path.join(data_directory, 'test_data')

#os.makedirs(train_directory, exist_ok=True)
#os.makedirs(test_directory, exist_ok=True)

#wav_files = glob.glob(data_directory + '/*.wav')
#train_files, test_files = train_test_split(wav_files, test_size=0.2, random_state=42) #80:20 split

# Move files to Train directory
#for file_path in train_files:
#    file_name = os.path.basename(file_path)
#    shutil.move(file_path, os.path.join(train_directory, file_name))

# Move files to Test directory
#for file_path in test_files:
#    file_name = os.path.basename(file_path)
#    shutil.move(file_path, os.path.join(test_directory, file_name))


In [5]:
from sklearn.model_selection import train_test_split
import glob

data_directory = '/content/drive/MyDrive/dataset'

In [6]:
import os
import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
from hmmlearn import hmm

In [7]:
#Create the Cepstrum manually
#Cepstrum: Derive the cepstrum of the audio files. (you need to code it manually)
def compute_cepstrum_manual(y, sr, n_fft=512):
    # Step 1: Compute Power Spectrum
    stft_mag = np.abs(librosa.stft(y, n_fft=n_fft))
    power_spectrum = stft_mag**2

    # Step 2: Take Logarithm
    log_power_spectrum = np.log(power_spectrum)

    # Step 3: Inverse Fourier Transform (IFFT)
    cepstrum = np.fft.irfft(log_power_spectrum, axis=0)

    # Step 4: Extract Cepstral Coefficients (e.g., first 13 coefficients)
    cepstral_coeffs = cepstrum[:13, :].T  # Transpose and select desired coefficients

    return cepstral_coeffs


In [30]:
#Given Code from Assignment


#/content/drive/MyDrive/dataset
#data_directory = '/content/drive/MyDrive/dataset'
#loaded_data = load_data(data_directory)
def compute_cepstrum_manual(y, sr, n_fft=512):
    stft_mag = np.abs(librosa.stft(y, n_fft=n_fft))
    power_spectrum = stft_mag**2
    log_power_spectrum = np.log(power_spectrum)
    cepstrum = np.fft.irfft(log_power_spectrum, axis=0)
    cepstral_coeffs = cepstrum[:13, :].T  # Select first 13 coefficients
    return cepstral_coeffs



def load_data(directory):
    data = {}
    for file in os.listdir(directory):
        if file.endswith(".wav"):
            digit = file.split('_')[0]
            y, sr = librosa.load(os.path.join(directory, file), sr=None)


            # Compute features (Make Spectogram, Mel Spectogram, Cepstrum and MFCC)
            # example:
            spectrogram = np.abs(librosa.stft(y, n_fft=512)) #2-1
            mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=512) #2-2
            cepstrum = compute_cepstrum_manual(y, sr)  # 2-3
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, center=False, n_fft = 512, hop_length = 15, win_length = 30) #2-4  #2-4

            if digit not in data:
                data[digit] = []
            data[digit].append(mfccs.T)  # Transposed to make it (time, feature). Replace mfccs with your feature and reshape accordingly
    return data



def load_test_data(directory):
    test_data = []
    labels = []
    for file in os.listdir(directory):
        if file.endswith(".wav"):
            digit = file.split('_')[0]
            y, sr = librosa.load(os.path.join(directory, file), sr=None)

            # COMPUTE FEATURES FOR TEST DATA HERE
            spectrogram = np.abs(librosa.stft(y, n_fft=512)) #2-1
            mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=512) #2-2
            cepstrum = compute_cepstrum_manual(y, sr)  # 2-3
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, center=False, n_fft = 512, hop_length = 15, win_length = 30) #2-4



            test_data.append(mfccs.T) # Transposed to make it (time, feature). Replace mfccs with your feature and reshape accordingly
            labels.append(digit)
    return test_data, labels


In [33]:
def train_hmms(data, n_components=5, covariance_type='diag'):
    models = {}
    for digit, features in data.items():
        model = hmm.GaussianHMM(n_components=2, n_iter=10, covariance_type='diag', params='mct', init_params='mc')
        features = np.concatenate(features)
        model.fit(features)
        models[digit] = model
    return models


def test_model(models, test_data, labels):
    correct_predictions = 0
    total_predictions = len(labels)

    for features, label in zip(test_data, labels):
        best_score, best_digit = float('-inf'), None
        for digit, model in models.items():
            score = model.score(features)
            if score > best_score:
                best_score = score
                best_digit = digit
        if best_digit == label:
            correct_predictions += 1

    print(f"Total predictions: {total_predictions}")
    print(f"Correct predictions: {correct_predictions}")
    accuracy = correct_predictions / total_predictions
    return accuracy



# Load train data
train_data = load_data('/content/drive/MyDrive/dataset/train_data')
models = train_hmms(train_data)  # Pass the train_data variable

# Load test data
test_data, test_labels = load_test_data('/content/drive/MyDrive/dataset/test_data')  # Pass the test_files variable

accuracy = test_model(models, test_data, test_labels)
print(f"Model accuracy: {accuracy:.2f}")


Total predictions: 600
Correct predictions: 486
Model accuracy: 0.81


Ordered by Accuracy:

Total predictions: 600

Test 8:
Correct predictions: 190
Model accuracy: 0.32
mfccs = 2

Test 4:
Correct predictions: 368
Model accuracy: 0.61
mfccs = 5

Test 10:
Correct predictions: 374
Model accuracy: 0.62
mfccs = 250

Test 11:
Correct predictions: 375
Model accuracy: 0.62
mfccs = 100

Test 9:
Correct predictions: 379
Model accuracy: 0.63
mfccs = 200

Test 2:
Correct predictions: 385
Model accuracy: 0.64
mfccs =50

Test 12:
Correct predictions: 438
Model accuracy: 0.73
mfccs = 30

Test 3:
Correct predictions: 453
Model accuracy: 0.76
mfccs = 10

Test 5:
Correct predictions: 464
Model accuracy: 0.77
mfccs = 15

Test 13:
Correct predictions: 461
Model accuracy: 0.77
mfccs = 21

Test 4:
Correct predictions: 473
Model accuracy: 0.79
mfccs = 25

Test 7:
Correct predictions: 475
Model accuracy: 0.79
mfccs = 19

Test 1:
Correct predictions: 484
Model accuracy: 0.81
mfccs = 20
(hop_length = 256) = .79

Test 6:
Correct predictions: 495
Model accuracy: 0.82
mfccs = 18



The amount of correct predictions is dependent on MFCCs. The accuracy of the model varies due to how close the mfccs is close to 20. Test 1 has achieved the highest accuracy of the observed, getting 81%, and the lowest amount of accuracy is 32% with MCC being 2. If changed the MCC to 1, the accuracy would have been even lower.

The Accuracy for the model continues to drop the higher the mfcc gets. When the mfccs is 50, 200, 100, or 250 it shows that there isn't a point in going any higher than 50. The accuracy drops completely. It's being overfitted, and when the mfcc was 2, it was being underfitted.

Changing the FFT window did not seem to affect the accuracy. The values used were 128, 256, 512, 1024

For the best results, I believe consistently retraining the model when mfcc is between 18-22 would lead to the best result, with 20 most likely being the highest. This range offers a good compromise between complexity & accuracy.

I tested changing the fmin, where I changed it from 0 to 10. Doing so made the program less accurate on when mfcc = 20. I got a result of .76. Changing fmin to 20 made it even less accurate, going to .73

Setting fmin to 0, and fmax = 100, The accuracy dramatically got worse. Going to 0.23 accuracy. Setting fmax = 500 only increased it marginally to 0.36. This discouraged me from changing fmin & fmax at all as it only made the model less accurate.

Hop length affects the time resolution but has no noticeable effects when tested in this code. That isn't until it is paired with win_length. I did some tests where the hop length was half of win_length so there was good overlap between frames. And after training the data, there were no noticeable changes when the window & hop length were decently large. However, there was a significant increase in accuracy when the window length & hop length were short. With a window length of 30 & a hop length of 15, the accuracy got slightly better, going up 2-4%. This also made the time it took for the code to process, however, take around 2 minutes per try. This low win_length is useful when trying to recognize speech.

The center being true or false was tested, but it had no noticeable impact on the accuracy. Whether it was on or off, the level of accuracy was the same.